# TASK #1. DEFINE A LAMBDA FUNCTION

In [1]:
%%writefile lambda_function.py

import json
import uuid
 
def lambda_handler(event, context):
    
    # Read the input parameters
    count   = event['StockUnits']
    price   = event['StockPrice']
 
    # Calculate the total dollar value
    total =  count * price
 
    # Return the result
    return {
        'TotalFunds' :   total
            }


Writing lambda_function.py


In [3]:
# Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python
# Boto3 allows Python developer to write software that makes use of services like Amazon S3 and Amazon EC2
import boto3
import json 
client = boto3.client('lambda')

# You must zip up the code of the lambda function at some point
# so let's import ZipFile Module
# This module provides tools to create, read, write, append, and list a ZIP file.
from zipfile import ZipFile
with ZipFile('lambda.zip', 'w') as f:
    f.write('lambda_function.py')
    
# If submitting as a ZipFile, you need to insert raw data. 
with open('lambda.zip', 'rb') as f:
    zipped_code = f.read()
    
# Note that SageMaker Notebooks don't have access to lambda by default 
# So you need to give permission to this SageMaker Notebook to access lambda
response = client.create_function(
    FunctionName = 'FundsCalculator',
    Runtime = 'python3.8',
    Handler = 'lambda_function.lambda_handler',
    Code = dict(ZipFile=zipped_code),    
    Timeout = 60,
    MemorySize = 512,
    Role = 'arn:aws:iam::422132866096:role/service-role/My-First-Lambda-role-ozpjvowf'
)

# TASK #2. INVOKE A LAMBDA FUNCTION

In [5]:

# Obtain How many stocks and the number of stocks from the bank customer

stock_price = int(input('What is the unit price of the stock you would like to buy'))
stock_count = int(input('How many stocks units you would like to purchase?'))


response = client.invoke(
    FunctionName = 'FundsCalculator',
    InvocationType = 'RequestResponse',
    LogType = 'Tail',
    Payload = json.dumps({'StockUnits' : stock_count, 'StockPrice' : stock_price}).encode('utf-8'),
)


What is the unit price of the stock you would like to buy20
How many stocks units you would like to purchase?30


In [6]:
print(response)

{'ResponseMetadata': {'RequestId': 'c87fab28-d3d8-4822-98fc-0b7532c47bc6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 15 Feb 2022 22:20:05 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-amzn-requestid': 'c87fab28-d3d8-4822-98fc-0b7532c47bc6', 'x-amzn-remapped-content-length': '0', 'x-amz-executed-version': '$LATEST', 'x-amz-log-result': 'U1RBUlQgUmVxdWVzdElkOiBjODdmYWIyOC1kM2Q4LTQ4MjItOThmYy0wYjc1MzJjNDdiYzYgVmVyc2lvbjogJExBVEVTVApFTkQgUmVxdWVzdElkOiBjODdmYWIyOC1kM2Q4LTQ4MjItOThmYy0wYjc1MzJjNDdiYzYKUkVQT1JUIFJlcXVlc3RJZDogYzg3ZmFiMjgtZDNkOC00ODIyLTk4ZmMtMGI3NTMyYzQ3YmM2CUR1cmF0aW9uOiAyOS4xMCBtcwlCaWxsZWQgRHVyYXRpb246IDMwIG1zCU1lbW9yeSBTaXplOiA1MTIgTUIJTWF4IE1lbW9yeSBVc2VkOiA0MCBNQglJbml0IER1cmF0aW9uOiAyNzguMTYgbXMJCg==', 'x-amzn-trace-id': 'root=1-620c2715-4dca67ed3db9583a5a9b7c72;sampled=0'}, 'RetryAttempts': 0}, 'StatusCode': 200, 'LogResult': 'U1RBUlQgUmVxdWVzdElkOiBjODdmYWIyOC1kM2Q4LTQ4MjItOThmYy0wYjc1MzJjNDdiYzYgVmVyc2lvb

In [7]:
data = response['Payload'].read()

In [8]:
print(data)

b'{"TotalFunds": 600}'
